In [1]:
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer

2022-04-19 18:46:01.510607: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
class TextDataset(Dataset):
    def __init__(self, path, tokenizer, seq_length=512):
        with open(path) as f:
            data = f.read()
        tokens = tokenizer.encode(data)
        examples = []
        for i in range(0, len(tokens) - seq_length + 1, seq_length):
            examples.append(tokens[i:i + seq_length])
        self.samples = torch.LongTensor(examples)
        print('Loaded samples:', len(self.samples))
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return self.samples[item]

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")

404 Client Error: Not Found for url: https://huggingface.co/sberbank-ai/mGPT/resolve/main/config.json


In [4]:
dataset = TextDataset('data/sah.txt', tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)

Loaded samples: 2173


In [5]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT")

In [6]:
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [7]:
for epoch in range(3):
    print('Epoch', epoch)
    progressbar = tqdm(dataloader)
    losses = []
    for batch in progressbar:
        batch = batch.to(model.device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.detach().item())
        progressbar.set_description("Loss: %.3f" % np.mean(losses[-10:]))

Epoch 0


Loss: 2.281:   5%|▌         | 114/2173 [01:02<18:57,  1.81it/s]


KeyboardInterrupt: 